In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import regex
import string

In [2]:
med_list = pd.read_csv('Medicine Database.csv')

In [21]:
def get_details(med_list):

  medicine_url = med_list['Medicine URL']
  medicine_name = med_list['Product name']
  category = med_list['Category']
  composition = []    # empty list to store composition of all medicines
  manufacturer = []   # empty list to store manufacturer
  mrp = []            # empty list to store final price
  side_effects = []   # empty list to store side_effects
  uses = []           # empty list to store uses
  for url in medicine_url:
    response1 = requests.get(url)
    soup = BeautifulSoup(response1.text, 'html.parser')

    t2 = soup.find('div', {'class': 'product-detail'})
    if t2 is None:
      info = 'Data Not Found'
    else:
      info = (t2.text).strip().split('  ')

    t3 = soup.find('div', {'class': 'drug-con pull-left'})
    if t3 is None:
      info2 = 'Data Not Found'
    else:
      info2 = (t3.text).strip().split('  ')

    t4 = soup.find('span', {'class': 'final-price'})
    if t4 is None:
      info3 = '0'
    else:
      info3 = ((t4.text.split(':')[-1]).strip())[3:]

    composition.append((info[-1]).strip())
    manufacturer.append((info2[-2][5:]).strip())
    mrp.append(info3)

    use = soup.find('div', {'id':'np_tab1'})
    uses_ = []
    if use is None:
      uses_='Data Not Found'
    else:
      use = use.find_all('li')
      for u in use:
        uses_.append(u.text)
      uses_ = ', '.join(uses_)

    uses.append(uses_)

    se = soup.find('div', {'id':'np_tab5'})
    side_effects_ = []
    if se is None:
      side_effects_='Data Not Found'
    else:
      se = se.find('div', {'class':'subinner-cont active'})
      if se is None:
        side_effects_='Data Not Found'
      else:
        se = se.find_all('li')
        for s in se:
          side_effects_.append(s.text)
        side_effects_ = ', '.join(side_effects_)
      

    side_effects.append(side_effects_)
    

  med = pd.DataFrame()
  med['Medicine URL'] = medicine_url
  med['Medicine Name'] = medicine_name
  med['Category'] = category
  med['Composition'] = composition
  med['Manufacturer'] = manufacturer
  med['MRP'] = mrp
  med['Uses'] = uses
  med['Side Effects'] = side_effects
  return med

In [22]:
x = get_details(med_list.iloc[0:100])

In [23]:
x

,Medicine URL,Medicine Name,Category,Composition,Manufacturer,MRP,Uses,Side Effects
0,https://www.netmeds.com/prescriptions/atrest-1...,Atrest 12.5mg Tablet 10'S,ADHD,TETRABENAZINE 12.5MG,Centaur Pharmaceuticals Pvt Ltd,146.01,ATREST 12.5MG is used in the treatment of the ...,"depression, suicidal thoughts or actions, neur..."
1,https://www.netmeds.com/prescriptions/atrest-2...,Atrest 25mg Tablet 10'S,ADHD,TETRABENAZINE 25MG,Centaur Pharmaceuticals Pvt Ltd,261.49,ATREST 25MG is used in the treatment of the in...,"depression, suicidal thoughts or actions, neur..."
2,https://www.netmeds.com/prescriptions/capnea-i...,Capnea Injection 1ml,ADHD,Caffeine Citrate 20 MG,Cipla Ltd,272.25,It is used in the treatment of interrupted bre...,serious intestinal disease with bloody stools ...
3,https://www.netmeds.com/prescriptions/cognix-1...,COGNIX 120mg Tablet 10's,ADHD,GINKGO BILOBA 120 mg,Eris Lifesciences Ltd,128.70,Data Not Found,Data Not Found
4,https://www.netmeds.com/prescriptions/a-cn-gel...,A CN Gel(Topical) 20gm,Acne,CLINDAMYCIN 1%+NICOTINAMIDE 4%,Eskon Pharma,135.00,mild to moderate acne (spots),"dry skin, redness of the skin, skin burning or..."
...,...,...,...,...,...,...,...,...
95,https://www.netmeds.com/prescriptions/clindasi...,CLINDASIL A Gel 15gm,Acne,ADAPALENE 0.1 %+CLINDAMYCIN 1 %,Scott-Edil Pharmacia Ltd,135.00,Data Not Found,Data Not Found
96,https://www.netmeds.com/prescriptions/clindasu...,Clindasure 150mg Capsule 10'S,Acne,CLINDAMYCIN 150MG,Macleods Pharmaceuticals Pvt Ltd,135.20,It is used in the treatment of severe bacteria...,persistent or bloody diarrhoea (which may be a...
97,https://www.netmeds.com/prescriptions/clindato...,CLINDATOP GOLD Gel 20gm,Acne,CLINDAMYCIN 1 %+NICOTINAMIDE 4 %,GOLDEN ASTER PHARMA INC,70.00,Data Not Found,Data Not Found
98,https://www.netmeds.com/prescriptions/clindot-...,CLINDOT N Gel 20gm,Acne,CLINDAMYCIN 1 %+NICOTINAMIDE 4 %,Brinton Pharmaceuticals Ltd,187.00,Data Not Found,Data Not Found


In [25]:
medicine_database_detailed = get_details(med_list)

In [26]:
medicine_database_detailed.to_csv('medicine_database_detailed.csv')

In [27]:
from google.colab import files
files.download('medicine_database_detailed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
df = pd.read_csv('medicine_database_detailed.csv', engine='python', index_col =0)
df 

,Medicine URL,Medicine Name,Category,Composition,Manufacturer,MRP,Uses,Side Effects
0,https://www.netmeds.com/prescriptions/atrest-1...,Atrest 12.5mg Tablet 10'S,ADHD,TETRABENAZINE 12.5MG,Centaur Pharmaceuticals Pvt Ltd,146.01,ATREST 12.5MG is used in the treatment of the ...,"depression, suicidal thoughts or actions, neur..."
1,https://www.netmeds.com/prescriptions/atrest-2...,Atrest 25mg Tablet 10'S,ADHD,TETRABENAZINE 25MG,Centaur Pharmaceuticals Pvt Ltd,261.49,ATREST 25MG is used in the treatment of the in...,"depression, suicidal thoughts or actions, neur..."
2,https://www.netmeds.com/prescriptions/capnea-i...,Capnea Injection 1ml,ADHD,Caffeine Citrate 20 MG,Cipla Ltd,272.25,It is used in the treatment of interrupted bre...,serious intestinal disease with bloody stools ...
3,https://www.netmeds.com/prescriptions/cognix-1...,COGNIX 120mg Tablet 10's,ADHD,GINKGO BILOBA 120 mg,Eris Lifesciences Ltd,128.70,Data Not Found,Data Not Found
4,https://www.netmeds.com/prescriptions/a-cn-gel...,A CN Gel(Topical) 20gm,Acne,CLINDAMYCIN 1%+NICOTINAMIDE 4%,Eskon Pharma,135.00,mild to moderate acne (spots),"dry skin, redness of the skin, skin burning or..."
...,...,...,...,...,...,...,...,...
23561,https://www.netmeds.com/prescriptions/wound-fi...,Wound Fix Solution 100ml,Wound,HYPOCHLOROUS ACID 0.006%+OXIDIZED WATER 99.97%,Strides Shasun Ltd,140.00,Data Not Found,Data Not Found
23562,https://www.netmeds.com/prescriptions/wounsol-...,Wounsol Ointment 15gm,Wound,Povidone Iodine+Sucralfate,Theia Health Care Pvt Ltd,75.00,Data Not Found,Data Not Found
23563,https://www.netmeds.com/prescriptions/zerosept...,ZEROSEPTIC Ointment 15gm,Wound,POVIDINE IODINE 5 %,Helios Pharmaceuticals,75.00,Data Not Found,Data Not Found
23564,https://www.netmeds.com/prescriptions/ds-gold-...,DS GOLD ANTI AGEING Cream 50gm,Wrinkle/Anti,SKINCARE PREPARATION,Adonis Laboratories Pvt Ltd,999.00,Data Not Found,Data Not Found


In [20]:
len(df[df['Side Effects']=='Data Not Found'])

11920

In [21]:
df.iloc[0]

Medicine URL     https://www.netmeds.com/prescriptions/atrest-1...
Medicine Name                            Atrest 12.5mg Tablet 10'S
Category                                                      ADHD
Composition                                   TETRABENAZINE 12.5MG
Manufacturer                       Centaur Pharmaceuticals Pvt Ltd
MRP                                                         146.01
Uses             ATREST 12.5MG is used in the treatment of the ...
Side Effects     depression, suicidal thoughts or actions, neur...
Name: 0, dtype: object